# 09 - 종합 실습

## (1) 환경 준비

### 1) 라이브러리 불러오기

In [3]:
# 판다스, 넘파이 라이브러리 불러오기
import pandas as pd
import numpy as np

### 2) 데이터 불러오기

다음 세개의 데이터를 불러와서 저장합시다.

* sales : https://raw.githubusercontent.com/DA4BAM/dataset/master/sales.csv
* products : https://raw.githubusercontent.com/DA4BAM/dataset/master/products.csv
* customers : https://raw.githubusercontent.com/DA4BAM/dataset/master/customers2.csv

In [4]:
sales = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/sales.csv')
products = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/products.csv')
customers = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/customers2.csv')


세 데이터프레임의 상위 5개 행을 조회하시오.

In [5]:
sales.head()

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID
0,107,2,2016-01-02,p1036481,2,2100,c150417
1,69,1,2016-01-02,p1152861,1,1091,c212716
2,69,7,2016-01-02,p1013161,1,2600,c212716
3,69,8,2016-01-02,p1005771,1,1650,c212716
4,69,11,2016-01-02,p1089531,1,2600,c212716


In [6]:
products.head()

,ProductID,ProductName,Category,SubCategory,CategoryOrd
0,p1052661,새우깡,간식,과자,3
1,p1054261,고구마스틱,간식,과자,3
2,p1097821,짱구,간식,과자,3
3,p1097831,감자칩,간식,과자,3
4,p1119071,뿌셔뿌셔,간식,과자,3


In [7]:
customers.head()

,CustomerID,RegisterDate,Gender,BirthYear
0,c328222,2014-09-25,F,1960
1,c281448,2013-06-18,F,1974
2,c038336,2003-10-10,F,1968
3,c084237,2007-03-09,F,1982
4,c162600,2010-06-14,F,1978


## (2) 데이터 집계, 수정, 조회하기

### 1) sales의 ProductID 별 판매량(Qty)과 판매액(Amt)의 합계를 tmp에 저장하고 조회하시오.

In [8]:
sales.head()
tmp = sales.groupby(sales['ProductID'], as_index=False)[['Qty', 'Amt']].sum()
print(tmp)

   ProductID   Qty       Amt
0   p1001771  1055   3354827
1   p1002841   903  11011541
2   p1005621   906   2601703
3   p1005771  3963   7319963
4   p1005891  5194  10119037
..       ...   ...       ...
57  p1246581  2180   2456346
58  p1255281   809    955633
59  p1256521   701   1241545
60  p1284851  2350   6437323
61  p1299491   530   1094319

[62 rows x 3 columns]


### 2)  1번 결과를 판매액 기준 내림차순으로 정렬하고 상위 5개 상품을 조회하시오.  
(매출액 상위 5개 상품을 조회하시오.)

In [9]:
tmp.sort_values(by='Amt', ascending=False, inplace=True)
print(tmp.head())

   ProductID   Qty       Amt
22  p1072601  4058  18129067
43  p1178011  1653  14078818
1   p1002841   903  11011541
4   p1005891  5194  10119037
46  p1194801   990   7517664


### 3) customers의 성별(Gender) 별 고객 수를 조회하시오.

In [10]:
customers.groupby('Gender', as_index=False)['CustomerID'].count()

,Gender,CustomerID
0,F,2060
1,M,183


### 4) customers의 출생연도(BirthYear) 로 부터 나이(Age)를 계산해서 변수로 추가하시오.

In [11]:
customers['Age'] = 2022-customers['BirthYear']

### 5) customers의 나이(Age)를 나이대(AgeGroup)로 변환하시오.

* [ , 30)  : 30미만
* [30, 40) : 30대
* [40, 50) : 40대
* [50, 60) : 50대
* [60, 70) : 60대
* [70, )   : 70이상

> [30, 40) 의 의미 : 30 <= ,  < 40

In [12]:
bins = [0,30,40,50,60,70,np.inf]
labels = ['30미만', '30대', '40대', '50대', '60대', '70이상']
customers['AgeGroup'] = pd.cut(customers['Age'], bins=bins, labels=labels, right=False)
customers[customers['Age']==70].head()

,CustomerID,RegisterDate,Gender,BirthYear,Age,AgeGroup
204,c097385,2008-02-05,F,1952,70,70이상
335,c130317,2009-06-06,F,1952,70,70이상
527,c165917,2010-07-29,F,1952,70,70이상
556,c366892,2015-12-26,F,1952,70,70이상
583,c038272,2003-10-08,F,1952,70,70이상


### 6) customers의 나이대(AgeGroup) 별 고객수를 구해 봅시다.

In [13]:
customers.groupby('AgeGroup', as_index=False)['CustomerID'].count()

,AgeGroup,CustomerID
0,30미만,6
1,30대,186
2,40대,925
3,50대,749
4,60대,267
5,70이상,110


### 7) 매출액 top 5 상품명을 조회해 봅시다.

* 매출액은 sales에 있고, 상품명은 products에 있습니다. 한꺼번에 조회하려면 merge를 사용해야 합니다.
    * 1) sales와 products를 ProductID 기준으로 inner 조인을 합시다.
    * 2) 상품명 별 매출액 합계를 집계합니다. 
    * 3) 매출액 합계 기준 내림차순으로 정렬하고 top 5를 조죄합니다.


In [14]:
# 1) sales와 products를 ProductID 기준으로 inner 조인을 합시다.

sales2 = sales.merge(products, on='ProductID', how='inner')


In [15]:
# 2) 2) 상품명 별 매출액 합계를 집계합니다.
tmp = sales2.groupby('ProductName', as_index=False)['Amt'].sum()
print(tmp.head())

  ProductName      Amt
0       감귤컵푸딩   677154
1         감자칩  3676518
2       고구마스틱  1620986
3          깻잎  2930359
4        낱개캔디   479197


In [16]:
# 3) 매출액 합계 기준 내림차순으로 정렬하고 top 5를 조죄합니다.
tmp.sort_values(by='Amt', ascending=False).head()


,ProductName,Amt
37,우유1000,18129067
23,사과_소,14078818
9,딸기_대,11011541
7,두부_대,10119037
10,딸기_소,7517664


### 8) 연령대 별 매출액을 조회합시다.

* 매출액은 sales에 있고, 연령대(AgeGroup)은 customers에 있습니다. 한꺼번에 조회하려면 merge를 사용해야 합니다.


In [17]:
# 1) sales와 customers를  CustomerID 기준으로 inner 조인을 합시다.
join1 = sales.merge(customers, on='CustomerID', how='inner')
join1.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 70012 entries, 0 to 70011
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   OrderID       70012 non-null  int64   
 1   Seq           70012 non-null  int64   
 2   OrderDate     70012 non-null  object  
 3   ProductID     70012 non-null  object  
 4   Qty           70012 non-null  int64   
 5   Amt           70012 non-null  int64   
 6   CustomerID    70012 non-null  object  
 7   RegisterDate  70012 non-null  object  
 8   Gender        70012 non-null  object  
 9   BirthYear     70012 non-null  int64   
 10  Age           70012 non-null  int64   
 11  AgeGroup      70012 non-null  category
dtypes: category(1), int64(6), object(5)
memory usage: 6.5+ MB


In [18]:
# 2) AgeGroup 별 매출액 합계를 집계합니다.
join1.groupby('AgeGroup', as_index=False)['Amt'].sum()


,AgeGroup,Amt
0,30미만,167539
1,30대,11730152
2,40대,86042546
3,50대,70047957
4,60대,19286478
5,70이상,7724796


### 9) [심화]연령대 별, 상품 카테고리 매출 비중(%)을 조회합시다.

* 1) 세 데이터프레임을 merge 해야 합니다.
    * merge는 한번에 두개씩만 가능하므로, 두번 수행합니다.
* 2) for loop를 이용하여, 각 연령대에 대해서 다음의 작업을 반복 수행합니다. 
    * 하나의 연령대 값에 대해서, 
        * 상품 카테고리(Category)별 매출액 합계를 구합니다.
        * 연령대에 해당하는 전체 매출액으로 나눠서, 카테고리별 비율을 계산합니다.
        * 결과를 출력합니다.



In [19]:
# 1) 세 데이터프레임을 merge 
join3 = sales.merge(customers, on='CustomerID', how='inner').merge(products, on='ProductID', how='inner')
join3.head()

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,RegisterDate,Gender,BirthYear,Age,AgeGroup,ProductName,Category,SubCategory,CategoryOrd
0,107,2,2016-01-02,p1036481,2,2100,c150417,2010-03-03,F,1974,48,40대,순두부,반찬류,두부,1
1,197,5,2017-01-24,p1036481,1,1050,c150417,2010-03-03,F,1974,48,40대,순두부,반찬류,두부,1
2,81,5,2016-03-13,p1036481,1,945,c350776,2015-06-06,F,1983,39,30대,순두부,반찬류,두부,1
3,81,6,2016-03-13,p1036481,1,945,c350776,2015-06-06,F,1983,39,30대,순두부,반찬류,두부,1
4,23,13,2016-10-16,p1036481,1,1050,c350776,2015-06-06,F,1983,39,30대,순두부,반찬류,두부,1


In [28]:
# 2) for loop를 이용하여, 각 연령대에 대해서 다음의 작업을 반복 수행합니다
from collections import defaultdict
Dict = defaultdict(int)
for agegroup in join3['AgeGroup'].unique():
  total = 0
  for index, order in join3[join3['AgeGroup']==agegroup].iterrows():
    total+= order['Amt']
    category = order['Category']
    Dict[category]+=order['Amt']
  print(agegroup)
  print('total :',total)
  for i in Dict:
    Dict[i] = Dict[i]/total
    print(i, Dict[i])


40대
total : 86042546
반찬류 0.13811890224633752
유제품 0.2570454156482073
채소 0.263177161215104
간식 0.08399565489380102
과일 0.25766286599655014
30대
total : 11730152
반찬류 0.12920609537872163
유제품 0.2553046420068057
채소 0.28223975811883434
간식 0.06863168388573779
과일 0.26461790586028777
50대
total : 70047957
반찬류 0.1712456100497848
유제품 0.22999504832531578
채소 0.2847826994046344
간식 0.0573811605759135
과일 0.2565954959202865
60대
total : 19286478
반찬류 0.24239620998948644
유제품 0.16464629933962274
채소 0.3319901272167318
간식 0.03432871763217528
과일 0.2266386976717831
70이상
total : 7724796
반찬류 0.2696081867270294
유제품 0.10895642611744043
채소 0.31739418516555357
간식 0.025016457952898384
과일 0.2790248734903417
30미만
total : 167539
반찬류 0.31078894829375087
유제품 0.1346319899034023
채소 0.38477200767692993
간식 0.08087684071444828
과일 0.08893618217175399
